In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pointcloud as pc
import importlib
%config InlineBackend.figure_format = 'retina'

dcolors = plt.rcParams['axes.prop_cycle'].by_key()['color']


In [ ]:
importlib.reload(pc)
samples_A = [np.random.rand(100, 2) for _ in range(10)]
samples_B = [np.random.normal(0.5, 0.2, size=(100, 2)) for _ in range(10)]

result = pc.compare_treatments_with_voronoi(samples_A, samples_B, test="t-test",plot=True)

# Lecture material

## Introduce hypothesis testing

## Coin example

## Explain p-value

## Intraclass correlation

### Flower example

## A statistical study from images

### Setting the stage

#### Recap from last weeks lecture
We saw that a  cell culture can be reduced to point cloud
- Voronoi tesselation
- Area distribution 

In [ ]:
intensity_control = 5.5
intensity_treatment = 6.0

In [ ]:
importlib.reload(pc)

fig,ax = plt.subplots(2,3,figsize=(10,7))  
dpoints=pc.point_field(intensity=intensity_control,ax=ax[0,0],title='Control culture',seed=100,plot=True)
lpoints=pc.point_field(intensity=intensity_treatment,ax=ax[1,0], title='Treatment culture',seed=200,plot=True)

from scipy.spatial import Voronoi, voronoi_plot_2d
dvor = Voronoi(dpoints)
dA=pc.compute_region_areas(dvor)
dA = dA[~np.isnan(dA)]
# dA=dA[dA<1.5]

lims = [3,6]
voronoi_plot_2d(dvor,show_points=True, show_vertices=False,ax=ax[0,1])
ax[0,1].set_aspect('equal')
ax[0,1].set_xlim(lims)
ax[0,1].set_ylim(lims)
ax[0,1].set_title('Voronoi tesselation of points')

ax[0,2].hist(dA,range=[0,3],bins=50);
ax[0,2].set_title('Histogram of region areas')

lvor = Voronoi(lpoints)
lA=pc.compute_region_areas(lvor)
lA = lA[~np.isnan(lA)]
# lA=lA[lA<1.5]
voronoi_plot_2d(lvor,show_points=True, show_vertices=False,ax=ax[1,1])
ax[1,1].set_aspect('equal')
ax[1,1].set_xlim(lims)
ax[1,1].set_ylim(lims)
ax[1,1].set_title('Voronoi tesselation of points')

ax[1,2].hist(lA,range=[0,3],bins=50);
ax[1,2].set_title('Histogram of region areas')
plt.tight_layout()

## Comparing samples

### Use the ICC to compare two samples

In [ ]:
importlib.reload(pc)
m=np.array([lA.mean(),dA.mean()])
s=np.array([lA.std(),dA.std()])

pc.icc(m,s)

#### Not very separated

With a ICC of this magnitude we can conclude that the two cultures are not differing very much. Let's see what our hypothesis testing tells us. This can naturally also be confirmed by looking at the histograms; they are more or less overlapping. 

### Can the t-test help us?

Let's try the t-test on the region areas 

$\mathcal{H}_0$: There is no difference between the cultures in the region area


In [ ]:
from scipy.stats import ttest_ind

stat, p_value = ttest_ind(dA, lA, equal_var=False)

print(f"p-value: {p_value:.4f}")

We confirm the null-hypothesis that there is no difference between the two samples. 

#### Outliers biasing the hypothesis test
We observe that there are some extreme outliers in the data. Let's prune...

In [ ]:
lAc=lA[lA<2]
dAc=dA[dA<2]

In [ ]:

fig,ax=plt.subplots(1,2,figsize=(12,4))
ax[0].hist(lA,range=[0,1.5],bins=50,alpha=0.5);
ax[0].axvline(np.mean(lA),color=dcolors[0])
ax[0].hist(dA,range=[0,1.5],bins=50,alpha=0.5);
ax[0].axvline(np.mean(dA),color=dcolors[1]);
ax[0].set_title('All data points')

ax[1].hist(lAc,range=[0,1.5],bins=50,alpha=0.5);
ax[1].axvline(np.mean(lAc),color=dcolors[0])
ax[1].hist(dAc,range=[0,1.5],bins=50,alpha=0.5);
ax[1].axvline(np.mean(dAc),color=dcolors[1]);
ax[1].set_title('Pruned data points');

In [ ]:
stat, p_value = ttest_ind(dAc, lAc, equal_var=False)

print(f"p-value: {p_value:.4f}")

#### What’s Really Happening?

Outliers increase variance, and the t-test depends on the ratio:

$$
\displaystyle
t=\frac{\text{difference in means}}{\text{pooled standard error}}
$$
 
→ Higher variance → larger denominator → smaller t → higher p-value

So outliers can dilute the test’s power and hide real effects.

#### Key Takeaway
Outliers can obscure true group differences. But you shouldn’t blindly remove them — it depends on context. In this particular case we know that voronoi regions at the boundary are infinite of very large. What we should have done here is to crop the regions at the image boundary. The cropping would of course also introduce an error. So, it is important to decide wether you can accept reducing the number of points or if the cropping is acceptable. 

### The Kolmogorov-Smirnov test

In statistics, the Kolmogorov–Smirnov test (also K–S test or KS test) is a nonparametric test of the equality of continuous one-dimensional probability distributions. 

It can be used to test whether 
- a sample came from a given reference probability distribution (one-sample K–S test), or
- two samples came from the same distribution (two-sample K–S test)

Intuitively, it provides a method to answer the question "How likely is it that we would see a collection of samples like this if they were drawn from that probability distribution?" or, in the second case, "How likely is it that we would see two sets of samples like this if they were drawn from the same (but unknown) probability distribution?"

In [ ]:
importlib.reload(pc)
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ks_2samp
dcolors = plt.rcParams['axes.prop_cycle'].by_key()['color']

# Generate two synthetic samples
np.random.seed(42)
N=2000
sample_A = np.random.normal(0, 1, N)
sample_B = np.random.normal(0.1, 1, N)  # Shifted mean
sample_C = np.random.normal(0.5, 1, N)  # Shifted mean

fig,ax = plt.subplots(1,2,figsize=(12,4))

pc.plot_ks_test(sample_A,sample_B,ax[0])
pc.plot_ks_test(sample_A,sample_C,ax[1])




### Use the KS test to compare two cultures

In [ ]:
pc.plot_ks_test(lA,dA,xlim=[0,1])

## Comparing treatments

Comparing one sample from each treatment is not sufficient
- There are natural variations in each sample
- The test outcome relies on luck like tossing a coin

A scientific requires many samples from each treatment.

Revise our hypothesis:
$\mathcal{H}_0$: Do treatment A and treatment B produce systematically different spatial patterns across replicates

But this time increase the population

### Produce data for the example

We generate two populations
- Gaussian distribution of the sample point intensity
- 10 samples in each population

In [ ]:
from scipy.stats import norm

# Define range and parameters
x = np.linspace(2, 10, 500)
mu = [5.75, 6]      # Mean
sigma = [1,1]   # Standard deviation

# Compute Gaussian PDF
y0 = norm.pdf(x, mu[0], sigma[0])
y1 = norm.pdf(x, mu[1], sigma[1])

# Plot
plt.figure(figsize=(8, 4))
plt.plot(x, y0, label=r'Control $\mathcal{N}$('+'{0:0.2f}, {1:0.1f})'.format(mu[0],sigma[0]), lw=2)
plt.plot(x, y1, label=r'Treatment $\mathcal{N}$'+'({0:0.2f}, {1:0.1f})'.format(mu[1],sigma[1]), lw=2)
plt.title('Standard Gaussian Bell Curve')
plt.xlabel('x')
plt.ylabel('Probability Density')
plt.grid(True)
plt.legend()
plt.tight_layout()

Can we separate them using the area of the Voronoi regions?

### Generate the points
1. Generate the point clouds
2. Compute Voronoi region areas

In [ ]:
importlib.reload(pc)
nSamples = 10
np.random.seed(seed=42)
samples_control   = np.random.normal(5.75, 1, size=nSamples) 
samples_treatment = np.random.normal(6.0, 1, size=nSamples)
width, height = 10,10
threshold=1.5

def compute_area_in_samples(samples, width,height, threshold) :
    A = []
    points = []
    for intensity in samples : 
        points.append(pc.point_field(intensity=intensity,width=width,height=height)) 
        a=pc.clipped_voronoi_areas(points[-1], bounding_box=(0, width, 0, height))
        a=a[a<threshold]
        A.append(a)
    return A,points

A_control,p_control     = compute_area_in_samples(samples_control, width=width,height=height,threshold=threshold)
A_treatment,p_treatment = compute_area_in_samples(samples_treatment, width=width,height=height,threshold=threshold)

In [ ]:
nPanels = 5
fig,axes=plt.subplots(2,nPanels,figsize=(15,6))

for ax,p in zip(axes[0],p_control[:nPanels]) :
    ax.scatter(p[:,1],p[:,0],s=0.5)
    ax.set_aspect('equal')
    
axes[0,0].set_ylabel('Control')
    
for ax,p in zip(axes[1],p_treatment[:nPanels]) :
    ax.scatter(p[:,1],p[:,0],s=0.5)
    ax.set_aspect('equal')

axes[1,0].set_ylabel('Treatment');

### Compare the populations

In [ ]:
stat_func = np.mean
avg_control   = np.array([stat_func(s) for s in A_control])
avg_treatment = np.array([stat_func(s) for s in A_treatment])

stat, p_value = ttest_ind(avg_control, avg_treatment, equal_var=False)

In [ ]:
nBins = 5
plt.hist(avg_control,bins=nBins,alpha=0.5,range=[0.1,0.3], label='Control')
plt.hist(avg_treatment,bins=nBins,alpha=0.5,range=[0.1,0.3], label='Treatment');
plt.xlabel('Average region area')
plt.ylabel('Counts');
plt.legend()
plt.title('p-value={0:0.3f}'.format(p_value));

Conclusion: given the data our t-test tells us to reject the null-hypothesis

### Let's try the KS-test

In [ ]:
importlib.reload(pc)
pc.plot_ks_test(avg_control,avg_treatment,xlim=[0,1])

#### When is the KS-test reliable?

We saw in this example that the KS-test delivered a very different p-value compared with the t-test. The following table shows when it makes sense to use the KS-test. In our case we only had ten samples to compare.

| Sample Size | Suitability for KS Test |
|---|:--|
|< 20|Use with caution; very low power|
|20–50|Can work for large or obvious differences|
|≥ 50|Reasonable reliability for moderate differences|
|≥ 100|Good power and reliable p-values|
|≥ 1000|Very sensitive — may detect tiny, even irrelevant differences|

In [ ]:
importlib.reload(pc)
vSamples=[5,10,20,50,100,200]
pvals = pc.test_samples_pvalues(vSamples =vSamples, seed=42) 

In [ ]:
plt.semilogy(vSamples,pvals)
plt.gca().set(title="p-values study", xlabel='Number of samples in the study', ylabel='p-value')
plt.savefig('figures/samples_vs_pvals.png',dpi=300)

In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import norm

# Define range and parameters
x = np.linspace(2, 10, 500)
mu = [5.75, 6]      # Mean
sigma = [1,1]   # Standard deviation

# Compute Gaussian PDF
y0 = norm.pdf(x, mu[0], sigma[0])
y1 = norm.pdf(x, mu[1], sigma[1])

# Plot
plt.figure(figsize=(8, 4))
plt.plot(x, y0, label=r'Control $\mathcal{N}$('+'{0:0.2f}, {1:0.1f})'.format(mu[0],sigma[0]), lw=2)
plt.plot(x, y1, label=r'Treatment $\mathcal{N}$'+'({0:0.2f}, {1:0.1f})'.format(mu[1],sigma[1]), lw=2)
plt.title('Standard Gaussian Bell Curve')
plt.xlabel('x')
plt.ylabel('Probability Density')
plt.grid(True)
plt.legend()
plt.tight_layout()